In [1]:
from keras import backend as K
import tensorflow as tf

from SSD_generate_anchors import generate_ssd_priors
from CustomDataLoader import DataAugmentor
from utility import *
from pathlib import Path
import collections
from ssd300_model import SSD300
from skimage.io import imread, imshow
import cv2
import numpy as np
from ssd_loss_function import SSDLoss



Using TensorFlow backend.


In [2]:
# root = Path.home()/'data'/'VOCdevkit'/'VOC2007'
root  = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'/'VOC2007'
voc_2007_datafile = root/'ImageSets'/'Main'/'train.txt'
voc_2007_images   = root/'JPEGImages'
voc_2007_annotations = root/'Annotations'



In [3]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]


In [4]:
priors = generate_ssd_priors(specs)
num_priors = priors.shape[0]

In [5]:
batch_size = 10

testloader = DataAugmentor()

data       = testloader.flow_from_directory(root = root,
                                            data_file=voc_2007_datafile,
                                            target_size=300,
                                            batch_size = batch_size,
                                            shuffle    = True
                                            )

sample = data[0]

images, targets = sample

In [6]:
loc_t  = np.zeros(shape = (batch_size, num_priors, 4), dtype=np.float32)
conf_t = np.zeros(shape = (batch_size, num_priors), dtype=np.float32)

In [7]:
print(loc_t.shape)
print(conf_t.shape)

(10, 8732, 4)
(10, 8732)


In [8]:
for idx in range(batch_size):
    truths = targets[idx][:,1:]

    labels = targets[idx][:,0]
    match(threshold= 0.5, 
          truths   = truths, 
          labels   = labels, 
          priors   = priors, 
          variance = [0.1, 0.2], 
          idx      = idx,
          loc_t    = loc_t,
          conf_t   = conf_t)

In [9]:
pos = conf_t > 0

In [10]:
pos.shape

(10, 8732)

In [11]:
num_pos = np.sum(pos, axis=1, keepdims= True)
print(num_pos.shape)

(10, 1)


In [12]:
num_pos

array([[ 6],
       [ 9],
       [39],
       [33],
       [22],
       [ 9],
       [63],
       [ 7],
       [24],
       [20]])

In [13]:
pos_idx = pos.reshape(pos.shape[0], pos.shape[1], 1).repeat(4, axis=-1)
print(pos_idx.shape)

(10, 8732, 4)


In [14]:
loc_data = np.random.randn(batch_size, 8732, 4)
print(loc_data.shape)

(10, 8732, 4)


In [15]:
loc_p = loc_data[pos_idx].reshape(-1,4)
loc_t = loc_t[pos_idx].reshape(-1,4)

In [16]:
print(loc_p.shape)
print(loc_t.shape)

(232, 4)
(232, 4)


In [18]:
print(conf_t.shape)
# batch_conf = conf_t.reshape(-1, 21)
# print(batch_conf.shape)

(10, 8732)
